In [1]:
# import torch
import pandas as pd
import tqdm

In [2]:

df = pd.read_csv(
    'data/Merged_Tweets_Sentiments.csv')

In [3]:
df.head()

,text,sentiment,polarity,subjectivity,Preprocess
0,This is the best summary you will find of Seas...,Pos,0.250000,0.366667,this is the best summary you will find of seas...
1,More tragic than the red wedding # got # gameo...,Neg,-0.083333,0.416667,more tragic than the red wedding got gameofthr...
2,Check out this awesome @ GameOfThrones Final S...,Pos,0.500000,0.833333,check out this awesome gameofthrones final sea...
3,If u saw this just know i wrote it just to see...,Pos,1.000000,0.700000,if saw this just know wrote it just to see the...
4,Now they just need 3 people,Neu,0.000000,0.000000,now they just need 3 people


In [4]:
df.sentiment.value_counts()

Neu    1863
Neg     570
Pos     557
Name: sentiment, dtype: int64

In [5]:
label_dict = {l:i for i,l in enumerate(df.sentiment.unique())}
num2label = {v:k for k, v in label_dict.items()}

In [6]:
num2label

{0: 'Pos', 1: 'Neg', 2: 'Neu'}

In [7]:

label_dict

{'Pos': 0, 'Neg': 1, 'Neu': 2}

In [8]:
df['label'] = df.sentiment.replace(label_dict)
df.head()

,text,sentiment,polarity,subjectivity,Preprocess,label
0,This is the best summary you will find of Seas...,Pos,0.250000,0.366667,this is the best summary you will find of seas...,0
1,More tragic than the red wedding # got # gameo...,Neg,-0.083333,0.416667,more tragic than the red wedding got gameofthr...,1
2,Check out this awesome @ GameOfThrones Final S...,Pos,0.500000,0.833333,check out this awesome gameofthrones final sea...,0
3,If u saw this just know i wrote it just to see...,Pos,1.000000,0.700000,if saw this just know wrote it just to see the...,0
4,Now they just need 3 people,Neu,0.000000,0.000000,now they just need 3 people,2


In [9]:
from sklearn.model_selection import train_test_split

ImportError: No module named 'sklearn.__check_build._check_build'
___________________________________________________________________________
Contents of C:\Users\lahir\Anaconda3\lib\site-packages\sklearn\__check_build:
setup.py                  _check_build.cp37-win_amd64.pyd__init__.py
__pycache__
___________________________________________________________________________
It seems that scikit-learn has not been built correctly.

If you have installed scikit-learn from source, please do not forget
to build the package before using it: run `python setup.py install` or
`make` in the source directory.

If you have used an installer, please check that it is suited for your
Python version, your operating system and your platform.

In [10]:
SEED = 42069

In [17]:

Xt, Xv, yt, yv = train_test_split(
    df.index.values,
    df.label.values,
    test_size=0.15,
    random_state=SEED,
    stratify=df.label.values
)

In [18]:
df['type'] = ['tmp'] * df.shape[0]

In [19]:
df.head()


,text,sentiment,polarity,subjectivity,Preprocess,label,type
0,This is the best summary you will find of Seas...,Pos,0.250000,0.366667,this is the best summary you will find of seas...,0,tmp
1,More tragic than the red wedding # got # gameo...,Neg,-0.083333,0.416667,more tragic than the red wedding got gameofthr...,1,tmp
2,Check out this awesome @ GameOfThrones Final S...,Pos,0.500000,0.833333,check out this awesome gameofthrones final sea...,0,tmp
3,If u saw this just know i wrote it just to see...,Pos,1.000000,0.700000,if saw this just know wrote it just to see the...,0,tmp
4,Now they just need 3 people,Neu,0.000000,0.000000,now they just need 3 people,2,tmp


In [20]:
df.loc[Xt, 'type'] = 'train'
df.loc[Xv, 'type'] = 'val'

In [21]:
df.groupby(['sentiment', 'label', 'type']).count()

text  polarity  subjectivity  Preprocess
sentiment label type                                           
Neg       1     train   485       485           485         485
                val      85        85            85          85
Neu       2     train  1583      1583          1583        1583
                val     280       280           280         280
Pos       0     train   473       473           473         473
                val      84        84            84          84

In [11]:

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

ModuleNotFoundError: No module named 'dataclasses'